In [ ]:
!pip install -q --upgrade pip
!pip install -q \
  transformers peft accelerate bitsandbytes sentencepiece einops gdown \
  datasets rouge_score sentence-transformers tqdm torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 159.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 199.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 214.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 181.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 170.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 202.1 MB/s eta 0:00

In [ ]:
import os, json, random, torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from datasets import Dataset, DatasetDict
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

# Constants
MODEL_NAME    = "Qwen/Qwen3-8B"
PEFT_MODEL_ID = "mohammad-shirkhani/Qwen3_finetune_follow_up_question"
DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LEN       = 512
BATCH_SIZE    = 8
MAX_GEN       = 50
TEMP, TOP_P   = 0.6, 0.85

# Data directory
data_dir = "/content/data"
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# 1) Load tokenizer and ensure left-padding
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, use_fast=True, trust_remote_code=True
)
tokenizer.pad_token     = tokenizer.eos_token
tokenizer.padding_side  = "left"

# 2) Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 3) Apply PEFT LoRA weights
peft_model = PeftModel.from_pretrained(base_model, PEFT_MODEL_ID)
peft_model.to(DEVICE)
peft_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/349M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 4096)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
prompt = (
    "Question: Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?\n"
    "Evidence: Jonathan Stark, a former US professional tennis player, won two Grand Slam doubles titles.\n"
    "Follow-up Question:"
)
inputs  = tokenizer(prompt, return_tensors="pt").to(DEVICE)
outputs = peft_model.generate(
    **inputs,
    max_new_tokens=30,
    temperature=0.1
)
# Post-processing
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Split on all occurrences of "Follow-up Question:"
prefix, *rest = decoded.split("Follow-up Question:")

if rest:
    # The first part after "Follow-up Question:"
    first_follow = rest[0].strip()
    # Build the final string: question and evidence + only the first Follow-up Question
    final_output = prefix.strip() + "\nFollow-up Question: " + first_follow
else:
    final_output = decoded

print(final_output)


Question: Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?
Evidence: Jonathan Stark, a former US professional tennis player, won two Grand Slam doubles titles.
Follow-up Question: How many Grand Slam titles did Henri Leconte win?


In [ ]:
# 1) Download dataset if needed
file_id  = "1qAVZOlyLXk4XCjONcyIhCkU0Buc98c2-"
out_path = f"{data_dir}/follow-upQA.json"
if not os.path.exists(out_path):
    import gdown
    gdown.download(id=file_id, output=out_path, quiet=False)

# 2) Load raw JSON
with open(out_path) as f:
    raw = json.load(f)

# 3) Build prompt-answer records
def build_prompt(r):
    return f"Question: {r['question']}\nEvidence: {r['evidence']}\nFollow-up Question:"

records = [
    {"prompt": build_prompt(r), "answer": r['follow_up_question'].strip()}
    for r in raw
]

# 4) Shuffle and split (90% train, 10% validation)
random.shuffle(records)
split = int(0.9 * len(records))
train_ds = Dataset.from_list(records[:split])
val_ds   = Dataset.from_list(records[split:])

dataset = DatasetDict({"train": train_ds, "validation": val_ds})
print(f"Train size: {len(train_ds)}, Validation size: {len(val_ds)}")

Downloading...
From: https://drive.google.com/uc?id=1qAVZOlyLXk4XCjONcyIhCkU0Buc98c2-
To: /content/data/follow-upQA.json
100%|██████████| 33.2M/33.2M [00:00<00:00, 96.9MB/s]


Train size: 45000, Validation size: 5000


In [6]:
# 1) Set up metrics and sentence encoder
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
import numpy as np
from tqdm import tqdm

rouge = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
sbert = SentenceTransformer("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

# 2) Helper to keep only the first "Follow-up Question:"
def extract_first_followup(text: str) -> str:
    parts = text.split("Follow-up Question:")
    if len(parts) <= 1:
        return text.strip()
    # parts[0] is everything before the first label; parts[1] starts with the first question
    first_follow = parts[1].split("Follow-up Question:")[0].strip()
    return f"Follow-up Question: {first_follow}"

def postprocess(gen_ids, prompt_len):
    # decode generated tokens, remove newlines, and extract first follow-up question
    txt = tokenizer.decode(gen_ids[prompt_len:], skip_special_tokens=True)
    txt = txt.replace("\n", " ").strip()
    return extract_first_followup(txt)

# 3) Run generation on the validation set and collect candidates/references
candidates, references = [], []
for i in tqdm(range(0, len(val_ds), BATCH_SIZE), desc="Generating"):
    batch = val_ds[i : i + BATCH_SIZE]
    enc = tokenizer(batch['prompt'], return_tensors='pt', padding=True).to(DEVICE)
    with torch.no_grad():
        outs = peft_model.generate(
            **enc,
            max_new_tokens=MAX_GEN,
            temperature=TEMP,
            top_p=TOP_P,
            do_sample=True
        )
    for j, out in enumerate(outs):
        prompt_len = (enc['input_ids'][j] != tokenizer.pad_token_id).sum().item()
        cand = postprocess(out, prompt_len)
        candidates.append(cand)
        references.append("Follow-up Question: " + batch['answer'][j])

# 4) Show how many pairs we have and print a few examples
print(f"Evaluating {len(candidates)} generated questions against {len(references)} ground-truth questions.\n")
for idx in range(5):
    print(f"Example {idx+1}:")
    print("  Generated:", candidates[idx])
    print("  Ground-Truth:", references[idx])
    print()

# 5) Compute and print ROUGE and Cosine metrics
r1_vals, rL_vals, cos_vals = [], [], []
for ref, cand in tqdm(zip(references, candidates), total=len(references), desc="Scoring"):
    scores = rouge.score(cand, ref)
    r1_vals.append(scores['rouge1'].fmeasure)
    rL_vals.append(scores['rougeL'].fmeasure)
    emb_r = sbert.encode(ref, convert_to_tensor=True)
    emb_c = sbert.encode(cand, convert_to_tensor=True)
    cos_vals.append(util.cos_sim(emb_r, emb_c).item())

metrics = {
    'ROUGE-1 (F1)': np.mean(r1_vals),
    'ROUGE-L (F1)': np.mean(rL_vals),
    'Cosine Sim':  np.mean(cos_vals)
}

print("Validation metrics:")
for name, value in metrics.items():
    print(f"{name:>15}: {value:.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating: 100%|██████████| 625/625 [1:14:55<00:00,  7.19s/it]


Evaluating 5000 generated questions against 5000 ground-truth questions.

Example 1:
  Generated: Follow-up Question: What was the occupation of Vladimir Drinfeld? NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
  Ground-Truth: Follow-up Question: NA

Example 2:
  Generated: Follow-up Question: In which year was the best known work by the composer of La Cetra composed?
  Ground-Truth: Follow-up Question: In addition to La Cetra, what other notable compositions are attributed to Antonio Vivaldi?

Example 3:
  Generated: Follow-up Question: In which year was "The Village" released?
  Ground-Truth: Follow-up Question: Which year was "The Visit" directed by M. Night Shyamalan released?

Example 4:
  Generated: Follow-up Question: In what year did the Cincinnati Red Stockings join the National League?
  Ground-Truth: Follow-up Question: In what year did the Cincinnati Red Stockings join the National League?

Example 5:
  Generated: Follow-up Question: In what year was Borussi

Scoring: 100%|██████████| 5000/5000 [01:23<00:00, 59.83it/s]

Validation metrics:
   ROUGE-1 (F1): 0.5069
   ROUGE-L (F1): 0.4779
     Cosine Sim: 0.6464
